In [1]:
import random
import copy
import time

In [2]:
def add_neighbours(visited, neighbours, pheromone_graph):
    for node in visited:
        for edge in pheromone_graph[node]:
            if edge not in visited:
                neighbours.append((node, edge))

In [3]:
def traverse(i, pheromone_graph, alpha, beta, k):
    n = len(pheromone_graph)
    tree = []
    tree_weight = 0
    visited = [i]
    while len(visited) < k:
        neighbours = []
        add_neighbours(visited, neighbours, pheromone_graph)
        if len(neighbours) == 0:
            return ([], float('inf'))
        values = [pheromone_graph[v[0]][v[1]]['pheromones']**alpha / pheromone_graph[v[0]][v[1]]['weight']**beta for v in neighbours]
        chosen_neighbour = random.choices(neighbours, weights = values, k = 1)[0]
        tree.append(chosen_neighbour)
        tree_weight += pheromone_graph[chosen_neighbour[0]][chosen_neighbour[1]]['weight']
        visited.append(chosen_neighbour[1])
    
    if len(visited) != k:
        return ([], float('inf'))
    
    return tree, tree_weight

In [4]:
def aco(pheromone_graph_original, num_ants, rho, num_iters, theta, alpha, beta, k):
    start = time.time()
    n = len(pheromone_graph_original)
    best_tree = ([], float('inf'))
    
    for j in range(num_iters):
        pheromone_graph = copy.deepcopy(pheromone_graph_original)
        node_best_tree = ([], float('inf'))
        
        for i in range(n):
            trees = [traverse(i, pheromone_graph, alpha, beta, k) for _ in range(num_ants)]

            for edge in pheromone_graph.edges:
                pheromone_graph[edge[0]][edge[1]]['pheromones'] *= rho

            for tree, tree_weight in trees:
                delta = theta / tree_weight
                for branch in tree:
                    pheromone_graph[branch[0]][branch[1]]['pheromones'] += delta

            ants_best_tree = min(trees, key = lambda c: c[1])
            if node_best_tree[0] == [] or ants_best_tree[1] < node_best_tree[1]:
                node_best_tree = copy.deepcopy(ants_best_tree)

        if best_tree[0] == [] or node_best_tree[1] < best_tree[1]:
            best_tree = copy.deepcopy(node_best_tree)

    if best_tree[0] != []:
        print(best_tree[0], best_tree[1])
    else:
        print("Not possible to construct a sufficient tree")
        
    end = time.time()
    print(end - start)
    
    return best_tree